In [121]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import random

def read_blocks(datafile):
    blocks = []
    current_block = []

    with open(datafile, 'r') as f:
        for line in f:
            stripped_line = line.strip()
            if stripped_line == "":
                if current_block:
                    blocks.append(np.array(current_block))
                    current_block = []
            else:
                numbers = list(map(float, line.split()))
                if len(numbers) == 13:
                    current_block.append(numbers)

    if current_block:
        blocks.append(np.array(current_block))

    return blocks


def calc_distance(frame, mean):
    dist = 0
    for i in range(len(mean)):
        dist += (frame[i] - mean[i])**2
    return dist


def calc_mean(cluster):
    new_mean = [0 for _ in range(13)]
    for frame in cluster:
        for mfcc in range(len(frame)):
            new_mean[mfcc] += frame[mfcc]
    for mfcc in range(len(new_mean)):
        new_mean[mfcc] = new_mean[mfcc] / len(cluster)
    return new_mean


def calc_covariance(cluster, mean):
    output = [[0 for _ in range(13)] for _ in range(13)]
    if (len(cluster) == 0):
        return output

    # full covariance
    for i in range(13):
        for j in range(13):
            cov = 0
            for x in range(len(cluster)):
                cov += (cluster[x][i] - mean[i]) * (cluster[x][j] - mean[j])
            output[i][j] = cov / len(cluster)

    # distinct diagonal matrix
    # for i in range(13):
    #     cov = 0
    #     for x in range(len(cluster)):
    #         cov += (cluster[x][i] - mean[i])**2
    #     output[i][i] = cov / len(cluster)
        
    return output


def calc_probability(clusters):
    total = 0
    for c in clusters:
        total += len(c)

    probabilities = [0 for _ in range(len(clusters))]
    for i in range(len(clusters)):
        if (len(clusters[i]) == 0):
            continue
        probabilities[i] = (len(clusters[i])/total)
    return probabilities


def calc_likelihood(x, mean, covariance, probability):
    n = probability * multivariate_normal.pdf(x, mean=mean, cov=covariance, allow_singular=True)
    return n


def reduce_covariance(x, y, covariance):
    output = [[covariance[x][x], covariance[x][y]], [covariance[x][y], covariance[y][y]]]
    return output



In [123]:
def expectation_maximization(number_clusters, digit_blocks, probabilities, means, covariances):
    # do expectation maximization on single digit with 3 means / clusters -> phonemes
    max_iterations = 75
    for i in range(max_iterations):
        # total frames per digit
        count_frames = 0
    
        # reset phoneme clusters
        clusters = [[] for _ in range(number_clusters)]
    
        for block in digit_blocks:
            for frame in block:
                count_frames += 1
                # calculate likelihood of a digit being in each cluster
                likelihoods = [0 for _ in range(number_clusters)]
                for l in range(len(likelihoods)):
                    likelihoods[l] = calc_likelihood(frame, means[l], covariances[l], probabilities[l])
    
                max_likelihood = likelihoods[0]
                max_likelihood_index = 0
                for l in range(len(likelihoods)):
                    if likelihoods[l] > max_likelihood:
                        max_likelihood = likelihoods[l]
                        max_likelihood_index = l
    
                clusters[max_likelihood_index].append(frame)
    
        # recalculate cluster mean after each iteration
        for m in range(len(means)):
            if len(clusters[m]) > 0:
                means[m] = calc_mean(clusters[m])
    
        # recalculate cluster covariance after each iteration
        for c in range(len(covariances)):
            if len(clusters[c]) > 0:
                covariances[c] = calc_covariance(clusters[c], means[c])
    
        # recalculate probability after each iteration
        # probability is len(cluster) / total frames per digit
        probabilities = calc_probability(clusters)

    return probabilities, means, covariances



In [125]:
# separate blocks corresponding to a single digit
file = 'Train_Arabic_Digit.txt'
allblocks = read_blocks(file)

number_clusters = 3
number_mfcc = 13

digit_info = [[] for _ in range(10)]

for i in range(10):
    digit = i
    a = digit*660
    b = (digit+1)*660
    digit_blocks = allblocks[a:b]

    means = [[random.uniform(-2, 2) for _ in range(number_mfcc)] for _ in range(number_clusters)]
    covariances = [np.identity(13) for _ in range(number_clusters)] # covariances is a list of 2D arrays corresponding to cluster covariance
    probabilities = [0.5 for _ in range(number_clusters)]
    
    probabilities, means, covariances = expectation_maximization(number_clusters, digit_blocks, probabilities, means, covariances)
    digit_info[i].append(probabilities)
    digit_info[i].append(means)
    digit_info[i].append(covariances)



In [126]:
print(digit_info[0][1])

[[-1.5629882933361214, -2.736751121411479, 0.48356421166866037, -0.28304576931279846, -0.2134984770813395, -0.3513688016752388, 0.023203666647428203, -0.3025552254425836, 0.19386004994557415, -0.0013772007924641287, 0.18444828980622, -0.07160080583433015, 0.10653414933612451], [0.04758245552902223, -5.894489815127995, 0.8084962955477597, -0.48999937672874944, 0.45431566906044496, -0.09752845538391262, 0.25127757345178264, 0.24948161187888443, 0.2827832480450175, 0.21627759639430957, 0.19417333668978412, 0.23337300996480792, 0.08196096069698343], [3.158198348310813, -2.205681221414199, -0.8325454040336058, -1.7924566605849956, -1.260270877460796, -1.2395694123159706, -0.7020384229160742, -0.5134969425779687, 0.027481967164918623, -0.46163128351271326, -0.34052815971861755, -0.20954222664055697, -0.14461141430298738]]


In [132]:
from scipy.stats import multivariate_normal

def get_likelihood(x, mean, cov):
    n = multivariate_normal.pdf(x, mean=mean, cov=cov, allow_singular=True)
    return n

In [ ]:
digit_likelihood = []

file = 'Test_Arabic_Digit.txt'
test_blocks = read_blocks(file)

number_clusters = 3

all_predictions = []

for i in range(10):
    digit = i
    a = digit*220
    b = (digit+1)*220
    digit_blocks = test_blocks[a:b]

    digit_predictions = []

    for utterance in digit_blocks:
        max_utterance_likelihood = float('-inf')
        max_utterance_likelihood_digit = 0
    
        # find the likelihood that an utterance is each digit
        for d in range(10):
            likelihood_utterance_is_digit = 0
            
            for frame in utterance:
                max_cluster_likelihood = float('-inf')
                digit_probabilities = digit_info[d][0]
                digit_means = digit_info[d][1]
                digit_covariances = digit_info[d][2]
                
                for i in range(number_clusters):
                    covariance = digit_covariances[i]
                    mean = digit_means[i]
                    prob = digit_probabilities[i]
        
                    epsilon = 1e-10
                    lik = np.log(np.maximum(get_likelihood(frame, mean, covariance), epsilon))
                    li = np.log(prob) + lik
                    if (li > max_cluster_likelihood):
                        max_cluster_likelihood = li
                
                likelihood_utterance_is_digit += max_cluster_likelihood
                
            if (likelihood_utterance_is_digit > max_utterance_likelihood):
                max_utterance_likelihood = likelihood_utterance_is_digit
                max_utterance_likelihood_digit = d
                
        digit_predictions.append(max_utterance_likelihood_digit)
        
    all_predictions.append(digit_predictions)

print(len(all_predictions))
print(all_predictions[0])

In [ ]:
confusion_matrix = [[0 for _ in range(10)] for _ in range (10)]
for i in range(len(all_predictions)):
    for j in range(len(all_predictions[i])):
        p = all_predictions[i][j]
        confusion_matrix[i][p] += 1

for i in range(10):
    print(confusion_matrix[i])

In [ ]:
accuracy = [0 for _ in range(10)]
total_accuracy = 0
for i in range(10):
    t = confusion_matrix[i][i]
    total_accuracy += confusion_matrix[i][i]
    accuracy[i] = t/220
total_accuracy /= 220 * 10

In [ ]:
for i in range(10):
    print(f"{accuracy[i]:.2%}")
print("\n")
print(f"{total_accuracy:.2%}")